In [1]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (8.0, 10.0)

In [8]:
dataDir='/home/yuliang/code/Deeplab-Seg/Deeplab/datasets/coco'
dataType='val2017'
annFile='{}/person_keypoints_{}.json'.format(dataDir,dataType)

In [9]:
# initialize COCO api for instance annotations
coco=COCO(annFile)

loading annotations into memory...
Done (t=0.24s)
creating index...
index created!


In [10]:
# display COCO categories and supercategories
cats = coco.loadCats(coco.getCatIds())
nms=[cat['name'] for cat in cats]
print('COCO categories: \n{}\n'.format(' '.join(nms)))

nms = set([cat['supercategory'] for cat in cats])
print('COCO supercategories: \n{}'.format(' '.join(nms)))

COCO categories: 
person

COCO supercategories: 
person


In [11]:
# get all images containing given categories, select one at random
catIds = coco.getCatIds(catNms=['person']);
imgIds = coco.getImgIds(catIds=catIds);
img = coco.loadImgs(imgIds[np.random.randint(0,len(imgIds))])[0]
print(len(imgIds))

2693


In [ ]:
from tqdm import tqdm
txt_path = "/home/yuliang/code/AlphaPose-pytorch/train_sppe/data/coco/list/train2017.txt"
with open(txt_path, 'wb') as t:
    for imgid in tqdm(imgIds):
        t.write(("%012d\n"%imgid).encode())

In [ ]:
# load and display image
# I = io.imread('%s/images/%s/%s'%(dataDir,dataType,img['file_name']))
# use url to load image
I = io.imread(img['coco_url'])
plt.axis('off')
plt.imshow(I)
plt.show()

In [ ]:
# load and display instance annotations
plt.imshow(I); plt.axis('off')
annIds = coco.getAnnIds(imgIds=262145, catIds=["person"], iscrowd=None)
anns = coco.loadAnns(annIds)
coco.showAnns(anns)

In [ ]:
# initialize COCO api for person keypoints annotations
dataDir='/home/yuliang/code/Deeplab-Seg/Deeplab/datasets/coco'
dataType='train2017'
annFile='{}/person_keypoints_{}.json'.format(dataDir,dataType)
coco_kps=COCO(annFile)

In [2]:
import numpy as np
import multiprocessing
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool
import os
from PIL import Image
from tqdm import tqdm

# initialize COCO api for person keypoints annotations
dataDir='/home/yuliang/code/Deeplab-Seg/Deeplab/datasets/coco'
dataType='train2017'
annFile='{}/person_keypoints_{}.json'.format(dataDir,dataType)
coco_kps=COCO(annFile)

pool = Pool() 
# pool = ThreadPool(1000)

guassDir = "/home/yuliang/code/Deeplab-Seg/Deeplab/datasets/coco/PoseHeatmap/guass"
binaryDir = "/home/yuliang/code/Deeplab-Seg/Deeplab/datasets/coco/PoseHeatmap/binary"
catIds = coco_kps.getCatIds(catNms=['person']);
imgIds = coco_kps.getImgIds(catIds=catIds);

def create_heatmap(size, poses, sigma):
        
    x = np.arange(0, size[1]) #width
    y = np.arange(0, size[0]) #height
    x, y = np.meshgrid(x, y)
    guass_heatmap = np.zeros([poses.shape[1], size[0], size[1]])
    binary_heatmap = np.zeros([size[0], size[1]], dtype=np.uint8)
    sigma_x, sigma_y = sigma/size[1]*8.0, sigma/size[0]*8.0

    for pose in poses:
        for idx,(x0,y0,c0) in enumerate(pose):
            if c0>0:
                gap = 5
                guass_heatmap[idx] += np.exp(-(((x-x0)/size[1])**2/(2.0*sigma_x**2)+((y-y0)/size[0])**2/(2.0*sigma_y**2)))
                binary_heatmap[max(y0-gap,0):min(y0+gap,size[0]-1),max(x0-gap,0):min(x0+gap,size[1]-1)] = idx

    return guass_heatmap, binary_heatmap

# class ClockProcess(multiprocessing.Process):
#     def __init__(self, imgid):
#         multiprocessing.Process.__init__(self)
#         self.imgid = imgid

#     def run(self):
#         one_thread(self.imgid)
             
def one_thread(imgid):
    img = coco_kps.loadImgs(imgid)[0]
    annIds = coco_kps.getAnnIds(imgIds=img['id'], catIds=catIds, iscrowd=None)
    anns = coco_kps.loadAnns(annIds)
    all_keypoints = np.array([ann["keypoints"] for ann in anns]).reshape(-1,17,3)
    guass, binary = create_heatmap((img['height'], img['width']), all_keypoints, 1.0)
    np.save(os.path.join(guassDir, "%012d.npy"%imgid), guass)
    im = Image.fromarray(binary)
    im.save(os.path.join(binaryDir,"%012d.png"%imgid),"PNG")
    
    pbar.update(1)
    

loading annotations into memory...
Done (t=6.08s)
creating index...
index created!


Process ForkPoolWorker-3:
Process ForkPoolWorker-5:
Process ForkPoolWorker-4:
Process ForkPoolWorker-1:
Process ForkPoolWorker-2:
Process ForkPoolWorker-6:
Process ForkPoolWorker-7:
Process ForkPoolWorker-11:
Process ForkPoolWorker-10:
Process ForkPoolWorker-8:
Process ForkPoolWorker-9:
Process ForkPoolWorker-12:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-15:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-14:
Traceback (most recent call last):
  File "/home/yuliang/.pyenv/versions/3.6.2/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-13:
Traceback (most recent call last):
Process ForkPoolWorker-16:
Traceback (most recent call last):
  File "/home/yuliang/.pyenv/versions/3.6.2/lib/python3.6/multiprocessing/process.py", line 93, in ru

AttributeError: Can't get attribute 'one_thread' on <module '__main__'>
  File "/home/yuliang/.pyenv/versions/3.6.2/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/yuliang/.pyenv/versions/3.6.2/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
AttributeError: Can't get attribute 'one_thread' on <module '__main__'>
  File "/home/yuliang/.pyenv/versions/3.6.2/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/yuliang/.pyenv/versions/3.6.2/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/yuliang/.pyenv/versions/3.6.2/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/yuliang/.pyenv/versions/3.6.2/lib/python3.6/multiprocessing/queues.py", line 344, in get
    return _ForkingPickler.loads(res

In [3]:
# for imgid in tqdm(imgIds):
#     p = ClockProcess(imgid)
#     p.start()

pbar = tqdm(total=len(imgIds))
# for imgid in tqdm(imgIds):
#     pool.apply_async(one_thread, args=(imgid,))
pool.map(one_thread, sorted(imgIds))

# # pool.imap_unordered(one_thread, sorted(imgIds))
# pool.close()
# pool.join()
# pbar.close()

  0%|          | 1/64115 [00:26<468:57:11, 26.33s/it]Process ForkPoolWorker-27:
Traceback (most recent call last):
  File "/home/yuliang/.pyenv/versions/3.6.2/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/yuliang/.pyenv/versions/3.6.2/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/yuliang/.pyenv/versions/3.6.2/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/home/yuliang/.pyenv/versions/3.6.2/lib/python3.6/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "<ipython-input-2-48f19b1d0aee>", line 55, in one_thread
    np.save(os.path.join(guassDir, "%012d.npy"%imgid), guass)
  File "/home/yuliang/.pyenv/versions/3.6.2/envs/env3.6/lib/python3.6/site-packages/numpy/lib/npyio.py", line 511, in save
    pickle_kwargs=pickle_kwargs)
  File "/home/yuliang/.pyenv/versions/3.6.2/envs/en

KeyboardInterrupt: 

In [ ]:
plt.imshow(guass[3], alpha=1.0)

In [ ]:
plt.imshow(io.imread(img['coco_url']))

In [ ]:
# load and display keypoints annotations
from matplotlib.patches import Polygon
plt.imshow(I); plt.axis('off')
ax = plt.gca()
annIds = coco_kps.getAnnIds(imgIds=262145, catIds=catIds, iscrowd=None)
anns = coco_kps.loadAnns(annIds)
coco_kps.showAnns(anns)

In [ ]:
# initialize COCO api for caption annotations
annFile = '{}/annotations/captions_{}.json'.format(dataDir,dataType)
coco_caps=COCO(annFile)

In [ ]:
# load and display caption annotations
annIds = coco_caps.getAnnIds(imgIds=img['id']);
anns = coco_caps.loadAnns(annIds)
coco_caps.showAnns(anns)
plt.imshow(I); plt.axis('off'); plt.show()

In [3]:
from multiprocessing import Pool
import os,time,random

def long_time_task(name):
    print('Run task %s (%s)...' %(name,os.getpid()))
    start = time.time()
    time.sleep(random.random()*3)
    end = time.time()
    print ('Task %s runs %0.2f seconds.' %(name,(end-start)))

if __name__ == '__main__':
    print ('Parent process %s.' % os.getpid())
    p = Pool()
    for i in range(5):
        p.apply_async(long_time_task,args=(i,))
    print ('Waiting for all subprocesses done...')
    p.close()
    p.join()
    print ('All subprocesses done.')


Parent process 4560.
Run task 0 (7120)...
Run task 1 (7121)...
Run task 2 (7122)...
Run task 3 (7123)...
Run task 4 (7124)...
Waiting for all subprocesses done...
Task 4 runs 0.23 seconds.
Task 0 runs 1.05 seconds.
Task 1 runs 1.07 seconds.
Task 2 runs 2.27 seconds.
Task 3 runs 2.54 seconds.
All subprocesses done.
